In [ ]:
# import and check for TensorFlow version then install 2.0.0 if not installed
"""
!pip install --upgrade tensorflow-gpu
"""
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
print(tf.__version__)
#tf.debugging.set_log_device_placement(True)

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# install scikit-optimize
!pip install scikit-optimize

# install autotime to get execution time for each cell
!pip install ipython-autotime
%load_ext autotime

# install PyDrive
!pip install -U -q PyDrive

2.3.0


In [ ]:
print(tf.__version__)
if tf.executing_eagerly():
    print("Eager execution!")

2.3.0
Eager execution!
time: 2.44 ms


In [ ]:
# imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random as ran
import skopt
import os
from skopt import gp_minimize
from tensorflow.python.client import timeline
from datetime import datetime
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Add, Concatenate, Reshape, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, AveragePooling1D, AveragePooling2D, Conv1DTranspose, Conv2DTranspose, UpSampling1D, UpSampling2D
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import cifar10

# authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

time: 59 ms


In [ ]:
# seed
seed_value = 23

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
ran.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. Configure a new global `tensorflow` session
"""
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
"""

'\nsession_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)\nsess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)\nK.set_session(sess)\n'

time: 13.8 ms


In [ ]:
# install gpu utilities
!pip install gputil
!pip install psutil
!pip install humanize

# import packages
import os, sys, humanize, psutil, GPUtil

# GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError("GPU device not found!")
print('Found GPU at: {}'.format(device_name))

# define function
def gpu_report():
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
    GPUs = GPUtil.getGPUs()
    print(GPUs)
    for i, gpu in enumerate(GPUs):
        print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

# GPU memory check
gpu_report()

Found GPU at: /device:GPU:0
CPU RAM Free: 25.7 GB
GPU 0 ... Mem Free: 15921MB / 16280MB | Utilization   2%
time: 8.52 s


In [ ]:
# fuction to load data
def load_data(link, name):
    fluff, id = link.split('d/')
    id, _ = id.split('/view')
    print("--- loading data: " + name + " from id -> " + id + " ---")
    downloaded = drive.CreateFile({"id":id})
    downloaded.GetContentFile(name)
    data = np.load(name, allow_pickle=True)
    return data

time: 7.29 ms


In [ ]:
# links for data loading
link_data_wr_train = "https://drive.google.com/file/d/14P-Kje0uorg7vf_4_Mh2445nAodJufMn/view?usp=sharing"
link_data_sp_train = "https://drive.google.com/file/d/1mRBFj4eX-shxWsbjMGEQs5BOnymJB14B/view?usp=sharing"
link_labels_dgt_train = "https://drive.google.com/file/d/1ObYhK-6O22QkLnyPCMjuh4STDfIhuf5K/view?usp=sharing"
link_data_wr_test = "https://drive.google.com/file/d/1Yzh8kO5Fh9iz2-xNOvIZ4GLdVoge4Tcb/view?usp=sharing"
link_data_sp_test = "https://drive.google.com/file/d/1CC1EzMNCnwWF52oh7DFTGgxmmbEfHsav/view?usp=sharing"
link_labels_dgt_test = "https://drive.google.com/file/d/1ZlDG5iDSi6ebamwwYp9tjaEgcghJUqBN/view?usp=sharing"

link_data_dvs_train = "https://drive.google.com/file/d/1VHsmJAleqP9pp9Nty5ZP9DkNkOFRhyPf/view?usp=sharing"
link_data_emg_train = "https://drive.google.com/file/d/1-qHwORnLzr4-DJu84kx6D6LfWGnR7Ipm/view?usp=sharing"
link_labels_hnd_train = "https://drive.google.com/file/d/1PkSYnCUubNp2lbm5aL7Xcwl7UGW1Zj8l/view?usp=sharing"
link_data_dvs_test = "https://drive.google.com/file/d/1sSMX49D9GwT1isF31i8JJlD34PF7b3SA/view?usp=sharing"
link_data_emg_test = "https://drive.google.com/file/d/1EnmIMvGTLYeiS5aUCypsSXTKoPMWXJhc/view?usp=sharing"
link_labels_hnd_test = "https://drive.google.com/file/d/1KLr2KbX5GRltpKVf5LKh6Zshn3nietZG/view?usp=sharing"


# names for data loading
name_data_wr_train = "data_wr_train.npy"
name_data_sp_train = "data_sp_train.npy"
name_labels_dgt_train = "labels_dgt_train.npy"
name_data_wr_test = "data_wr_test.npy"
name_data_sp_test = "data_sp_test.npy"
name_labels_dgt_test = "labels_dgt_test.npy"

name_data_dvs_train = "data_dvs_train.npy"
name_data_emg_train = "data_emg_train.npy"
name_labels_hnd_train = "labels_hnd_train.npy"
name_data_dvs_test = "data_dvs_test.npy"
name_data_emg_test = "data_emg_test.npy"
name_labels_hnd_test = "labels_hnd_test.npy"

time: 12.8 ms


In [ ]:
# import dataset
"""
1 = written digits
2 = spoken digits
3 = dvs hand gestures
4 = emg hand gestures
"""
dataset = 4

if dataset == 1:
    class_nbr = 10
    train_data = 60000
    label_data = 6000
    test_data = 10000
    x_train = load_data(link_data_wr_train, name_data_wr_train)
    x_train = x_train.reshape((60000, 28, 28, 1))  # adapt this if using `channels_first` image data format
    index_train = load_data(link_labels_dgt_train, name_labels_dgt_train)
    x_test = load_data(link_data_wr_test, name_data_wr_test)
    x_test = x_test.reshape((10000, 28, 28, 1))  # adapt this if using `channels_first` image data format
    index_test = load_data(link_labels_dgt_test, name_labels_dgt_test)

if dataset == 2:
    class_nbr = 10
    train_data = 60000
    label_data = 6000
    test_data = 10000
    x_train = load_data(link_data_sp_train, name_data_sp_train)
    x_train = x_train.reshape((60000, 39, 13, 1))  # adapt this if using `channels_first` image data format
    index_train = load_data(link_labels_dgt_train, name_labels_dgt_train)
    x_test = load_data(link_data_sp_test, name_data_sp_test)
    x_test = x_test.reshape((10000, 39, 13, 1))  # adapt this if using `channels_first` image data format
    index_test = load_data(link_labels_dgt_test, name_labels_dgt_test)

if dataset == 3:
    class_nbr = 5
    train_data = 5400
    label_data = 540
    test_data = 1350
    x_train = load_data(link_data_dvs_train, name_data_dvs_train)
    x_train = x_train.reshape((5400, 60, 60, 1))  # adapt this if using `channels_first` image data format
    index_train = load_data(link_labels_hnd_train, name_labels_hnd_train)
    x_test = load_data(link_data_dvs_test, name_data_dvs_test)
    x_test = x_test.reshape((1350, 60, 60, 1))  # adapt this if using `channels_first` image data format
    index_test = load_data(link_labels_hnd_test, name_labels_hnd_test)

if dataset == 4:
    class_nbr = 5
    train_data = 5400
    label_data = 540
    test_data = 1350
    x_train = load_data(link_data_emg_train, name_data_emg_train)
    x_train = x_train.reshape((5400, 16, 1))  # adapt this if using `channels_first` image data format
    index_train = load_data(link_labels_hnd_train, name_labels_hnd_train)
    x_test = load_data(link_data_emg_test, name_data_emg_test)
    x_test = x_test.reshape((1350, 16, 1))  # adapt this if using `channels_first` image data format
    index_test = load_data(link_labels_hnd_test, name_labels_hnd_test)

--- loading data: data_emg_train.npy from id -> 1-qHwORnLzr4-DJu84kx6D6LfWGnR7Ipm ---
--- loading data: labels_hnd_train.npy from id -> 1PkSYnCUubNp2lbm5aL7Xcwl7UGW1Zj8l ---
--- loading data: data_emg_test.npy from id -> 1EnmIMvGTLYeiS5aUCypsSXTKoPMWXJhc ---
--- loading data: labels_hnd_test.npy from id -> 1KLr2KbX5GRltpKVf5LKh6Zshn3nietZG ---
time: 5.29 s


In [ ]:
class KSOM():
    def __init__(self, m, n, dim):
        self.m = m
        self.n = n
        self.dim = dim
        self.map_wgt =  tf.Variable(
                            tf.random.uniform(
                                shape = [m*n, dim],
                                minval = 0.0,
                                maxval = 1.0,
                                dtype = tf.float32,
                                seed = 23
                            )
                        )
        self.map_loc =  tf.constant(
                            np.array(
                                list(self.neuron_locs(m, n))
                            )
                        )

    def neuron_locs(self, m, n):
        # nested iterations over both dimensions to yield one by one the 2-d locations of the individual neurons in the SOM
        for i in range(m):
            for j in range(n):
                yield np.array([i,j], dtype=np.float32)
    
    def compute_winner(self, sample):
            self.sample = sample

            # compute the squared euclidean distance between the input and the neurons
            self.squared_distance = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_wgt, # [m*n, dim]
                                                tf.expand_dims(
                                                    self.sample, # [dim] -> [1, dim]
                                                    axis=0
                                                )
                                            )
                                        ), 
                                        axis=1
                                    )

            # compute the consine distance between the input and the neurons
            """
            self.squared_distance =  tf.subtract(
                                          1.0,
                                          tf.math.divide(
                                              tf.reduce_sum(
                                                  tf.multiply(
                                                      self.map_wgt, # [m*n, dim]
                                                      tf.expand_dims(
                                                          self.sample, # [dim] -> [1, dim]
                                                          axis=0
                                                      )
                                                  ),
                                                  axis=1
                                              ),
                                              tf.multiply(
                                                  tf.math.sqrt(
                                                      tf.reduce_sum(
                                                          tf.multiply(
                                                              self.map_wgt, # [m*n, dim]
                                                              self.map_wgt, # [m*n, dim]
                                                          ),
                                                          axis=1
                                                      )
                                                  ),
                                                  tf.math.sqrt(
                                                      tf.reduce_sum(
                                                          tf.multiply(
                                                              self.sample, # [dim]
                                                              self.sample, # [dim]
                                                          ),
                                                          axis=0
                                                      )
                                                  )
                                              )
                                          )
                                      )
            """
            
            # find the bmu's index
            self.bmu_idx =  tf.argmin(
                                    input=self.squared_distance, 
                                    axis=0
                                )
            
            # extract the bmu's 2-d location
            self.bmu_loc =  tf.gather(
                                self.map_loc, 
                                self.bmu_idx
                            )
    
    def update_network(self, epsilon, eta):
        # compute the squared manhattan distance between the bmu and the neurons
        self.bmu_distance_squares = tf.reduce_sum(
                                        tf.square(
                                            tf.subtract(
                                                self.map_loc, # [m*n, 2]
                                                tf.expand_dims(
                                                    self.bmu_loc, # [2] -> [1, 2]
                                                    axis=0
                                                )
                                            )
                                        ), 
                                        axis=1
                                    )

        # compute the neighborhood function
        self.neighbourhood_func = tf.exp(
                                      tf.negative(
                                          tf.math.divide(
                                              self.bmu_distance_squares,
                                              tf.multiply(
                                                  tf.square(
                                                      eta,
                                                  ),
                                                  2.0
                                              )
                                          )
                                      )
                                  )

        # compute the overall learning of each neuron
        self.learning = tf.multiply(
                            self.neighbourhood_func, 
                            epsilon
                        )
        
        # compute the difference between the neurons weights and the input
        self.delta_wgt =  tf.subtract(
                              tf.expand_dims(
                                  self.sample, # [dim] -> [1, dim]
                                  axis=0
                              ),
                              self.map_wgt, # [m*n, dim]
                          )

        # compute the weights update according to the learning and delta_wgt and update the weights
        tf.compat.v1.assign_add(
            self.map_wgt,
            tf.multiply(
                tf.expand_dims(
                    self.learning, # [m*n] -> [m*n, 1]
                    axis=-1
                ),
                self.delta_wgt # [m*n, dim]
            )
        )
    
    def get_weights(self):
        return self.map_wgt

    @tf.function
    def train(self, nbr_epochs, epsilon_i, epsilon_f, eta_i, eta_f, x_train):
        with tf.device('/device:gpu:0'):
            for epoch in tf.range(nbr_epochs):
                tf.print("---------- epoch", epoch + 1, "----------")

                # update the learning rate epsilon
                epsilon_t =  tf.multiply(
                                    epsilon_i,
                                    tf.pow(
                                        tf.math.divide(
                                            epsilon_f, 
                                            epsilon_i
                                        ),
                                        tf.cast(
                                            tf.math.divide(
                                                epoch,
                                                nbr_epochs - 1
                                            ), 
                                            dtype=tf.float32
                                        )
                                    )
                                )
                
                # update the gaussian neighborhood witdh eta
                eta_t =  tf.multiply(
                                  eta_i, 
                                  tf.pow(
                                      tf.math.divide(
                                          eta_f, 
                                          eta_i
                                      ),
                                      tf.cast(
                                          tf.math.divide(
                                              epoch,
                                              nbr_epochs - 1
                                          ), 
                                          dtype=tf.float32
                                      )
                                  )
                              )
                
                # shuffle the training dataset
                tf.random.shuffle(x_train)

                # bmu computing and network update for each sample
                for x_trn in x_train:
                    sample = tf.cast(x_trn, dtype=tf.float32)
                    self.compute_winner(sample)
                    self.update_network(epsilon_t, eta_t)

time: 92.3 ms


In [ ]:
def euclid_dist(x, y):
    d = np.linalg.norm(x - y, axis=1)
    return d

def cosine_dist(x, y):
    cosine_similarity =  np.divide(
                              np.sum(
                                  np.multiply(x, 
                                              y,
                                  ),
                                  axis=1
                              ),
                              np.multiply(
                                  np.sqrt(
                                      np.sum(
                                          np.multiply(x, 
                                                      x,
                                          ),
                                          axis=0
                                      )
                                  ),
                                  np.sqrt(
                                      np.sum(
                                          np.multiply(y, 
                                                      y
                                          ),
                                          axis=1
                                      )
                                  )
                              )
                          )
    
    d = 1.0 - cosine_similarity
    return d

time: 8.15 ms


In [ ]:
def labeling(label_data, class_nbr, weights, x_label, index_label, sigma):    
    dist_sum = np.zeros((len(weights), class_nbr))
    nbr_digits = np.zeros((class_nbr,))

    # accumulate the normalized gaussian distance for the labeling dataset
    for (x, y) in zip(x_label, index_label):
        nbr_digits[y] += 1
        dist_neuron = np.exp(-euclid_dist(x, weights)/sigma)
        dist_bmu = np.max(dist_neuron)
        for i, distn in enumerate(dist_neuron):
            dist_sum[i][y] += distn/dist_bmu

    # normalize the activities on the number of samples per class
    for i, dists in enumerate(dist_sum):
        dist_sum[i] = dists/nbr_digits

    # assign the neurons labels
    neuron_label = np.argmax(dist_sum, axis=1)
    #print("neurons labels = ")
    #print(neuron_label)

    return neuron_label

time: 10.8 ms


In [ ]:
def test(class_nbr, weights, x_test, index_test, neuron_label, sigma):
    neuron_index = np.zeros((len(x_test),), dtype=int)

    # calculate the BMUs for the test dataset
    for i, x in enumerate(x_test):
        dist_neuron = euclid_dist(x, weights)
        neuron_index[i] = np.argmin(dist_neuron)
    
    # compare the BMUs labels and the samples labels
    accuracy = 0
    for p, t in zip(neuron_index, index_test):
        if neuron_label[p] == t:
            accuracy += 1
    accuracy = (float(accuracy)/len(x_test))*100
    print("mnist test accuracy = %.2f" % accuracy)

    return accuracy

time: 6.9 ms


In [ ]:
def run_som(eps_i, eps_f, eta_i, eta_f, x_tr, x_lb, index_lb, x_ts, index_ts):
    print("\nhyper-parameters:   # eps_i = %f   # eps_f = %f   # eta_i = %f   # eta_f = %f" % (eps_i, eps_f, eta_i, eta_f))
    # train the network
    som = KSOM(map_wth, map_hgt, input_dim)
    som.train(nbr_epochs, eps_i, eps_f, eta_i, eta_f,x_tr)
    weights = som.get_weights().numpy()
    
    # label the network
    neuron_label = labeling(label_data, class_nbr, weights, x_lb, index_lb, sigma_kernel)

    # test the network
    accuracy = test(class_nbr, weights, x_ts, index_ts, neuron_label, sigma_kernel)

    return weights, accuracy

time: 7.85 ms


In [ ]:
# function to shuffle in unison
def unison_shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def run_labeling(weights, class_nbr, label_data, x_tr, index_tr, x_ts, index_ts):
    for i in range(10):
        x_tr, index_tr = unison_shuffle(x_tr, index_tr)
        x_lb = np.copy(x_tr[:label_data,:])
        index_lb = np.copy(index_tr[:label_data])
        # label the network
        neuron_label = labeling(label_data, class_nbr, weights, x_lb, index_lb, sigma_kernel)
        
        # test the network
        accuracy = test(class_nbr, weights, x_ts, index_ts, neuron_label, sigma_kernel)

time: 7.42 ms


In [ ]:
# function to normalize and standardize data
def norm_stand(data, data_mean, data_std, data_max, data_min):
    print("--- normalize and standardize data ---")
    
    data = data.astype(float)

    # standardize
    """
    if data_mean == None:
        data_mean = np.mean(data)
    if data_std == None:
        data_std = np.std(data) + 1e-15
    print("data_mean = ", data_mean)
    print("data_std = ", data_std)
    data -= data_mean
    data /= data_std
    """
    
    # normalize
    if data_max == None:
        data_max = np.max(data)
    if data_min == None:
        data_min = np.min(data) 
    print("data_max = ", data_max)
    print("data_min = ", data_min)
    for i in range(len(data)):
        data[i] = (data[i] - data_min) / (data_max - data_min)  
    
    print("---")
    print("new_data_max = ", np.max(data))
    print("new_data_min = ", np.min(data))
    print("new_data_mean = ", np.mean(data))
    print("new_data_std = ", np.std(data))
    print("\n")
    
    return data, data_mean, data_std, data_max, data_min

time: 12.5 ms


In [ ]:
kl_divergence = tf.keras.losses.kullback_leibler_divergence # loss = y_true * log(y_true / y_pred)

class KLDivergenceRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, weight=0.01, target=0.05):
        self.weight = weight
        self.target = target

    def __call__(self, inputs):
        mean_activities = K.mean(inputs, axis=0)
        loss = self.weight * K.sum(
            kl_divergence(self.target, mean_activities) +
            kl_divergence(1. - self.target, 1. - mean_activities)
        )
        return loss

kld_reg = KLDivergenceRegularizer(weight=1e-4, target=0.05)

time: 7.74 ms


In [ ]:
def weight_bound(fan_in, fan_out):
    bound = np.sqrt(
                np.divide(
                    6.0,
                    np.add(
                        fan_in,
                        fan_out
                    )
                )
            )
    return bound

time: 2.33 ms


In [ ]:
# standardize/normalize data
x_train, data_mean, data_std, data_max, data_min = norm_stand(x_train, None, None, None, None)
x_test, data_mean, data_std, data_max, data_min = norm_stand(x_test, data_mean, data_std, data_max, data_min)

--- normalize and standardize data ---
data_max =  7.334081930525158
data_min =  -1.2475608376565506
---
new_data_max =  1.0
new_data_min =  0.0
new_data_mean =  0.14489273815932827
new_data_std =  0.11593214487955045


--- normalize and standardize data ---
data_max =  7.334081930525158
data_min =  -1.2475608376565506
---
new_data_max =  0.9472715918106641
new_data_min =  0.0
new_data_mean =  0.1473066810263437
new_data_std =  0.11886099780142306


time: 36.6 ms


In [ ]:
# Custom topology with the best performance
# Accuracy on MNIST: 96.83 - 96.53 - 96.97
"""
ker_reg = 1e-4
act_reg = 1e-4 
features_layer = 3
input_dim = 4096

input_img = Input(shape=(28, 28, 1)) # adapt this if using `channels_first` image data format
e = Conv2D(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)
e = Conv2D(256, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)

encoded = MaxPooling2D((5, 5))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional

d = UpSampling2D((5, 5))(encoded)
d = Conv2DTranspose(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
decoded = Conv2DTranspose(1, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
"""

"\nker_reg = 1e-4\nact_reg = 1e-4 \nfeatures_layer = 3\ninput_dim = 4096\n\ninput_img = Input(shape=(28, 28, 1)) # adapt this if using `channels_first` image data format\ne = Conv2D(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)\ne = Conv2D(256, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)\n\nencoded = MaxPooling2D((5, 5))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional\n\nd = UpSampling2D((5, 5))(encoded)\nd = Conv2DTranspose(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)\ndecoded = Conv2DTranspose(1, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)\n"

time: 3.52 ms


In [ ]:
# Network to use
# Custom topology with the best performance
ker_reg = 1e-4
act_reg = 1e-4 

if dataset == 1: 
    # Accuracy: 96
    features_layer = 3
    input_dim = 4096
    
    input_img = Input(shape=(28, 28, 1))
    e = Conv2D(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)
    e = Conv2D(256, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)

    encoded = MaxPooling2D((5, 5))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional

    d = UpSampling2D((5, 5))(encoded)
    d = Conv2DTranspose(64, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
    decoded = Conv2DTranspose(1, (5, 5), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)

elif dataset == 2:
    # Accuracy: 50
    features_layer = 3
    input_dim = 4608
    
    input_img = Input(shape=(39, 13, 1))
    e = Conv2D(64, (6, 3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)
    e = Conv2D(256, (5, 3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)

    encoded = MaxPooling2D((5, 3))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional

    d = UpSampling2D((5, 3))(encoded)
    d = Conv2DTranspose(64, (5, 3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
    decoded = Conv2DTranspose(1, (6, 3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)

elif dataset == 3:
    # Accuracy: 45
    features_layer = 3
    input_dim = 12544
    
    input_img = Input(shape=(60, 60, 1))
    e = Conv2D(64, (7, 7), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)
    e = Conv2D(256, (6, 6), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)

    encoded = MaxPooling2D((7, 7))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional

    d = UpSampling2D((7, 7))(encoded)
    d = Conv2DTranspose(64, (6, 6), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
    decoded = Conv2DTranspose(1, (7, 7), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)

elif dataset == 4:
    # Accuracy: 45
    features_layer = 3
    input_dim = 1024
    
    input_img = Input(shape=(16, 1))
    e = Conv1D(64, (3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(input_img)
    e = Conv1D(256, (3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg), activity_regularizer=regularizers.l1(act_reg))(e)

    encoded = MaxPooling1D((3))(e) # at this point the representation is (4, 4, 256) i.e. 4096-dimensional

    d = UpSampling1D((3))(encoded)
    d = Conv1DTranspose(64, (3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)
    decoded = Conv1DTranspose(1, (3), activation='relu', kernel_regularizer=regularizers.l2(ker_reg))(d)

time: 165 ms


In [ ]:
# create the model
autoencoder = Model(input_img, decoded)
autoencoder.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 16, 1)]           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 64)            256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 12, 256)           49408     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 256)            0         
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 12, 256)           0         
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 14, 64)            49216     
_________________________________________________________________
conv1d_transpose_3 (Conv1DTr (None, 16, 1)            

In [ ]:
# compile it
# sgd, adam or adadelta?
autoencoder.compile(
    optimizer=tf.keras.optimizers.Adadelta( 
        learning_rate=1.0,
        rho=0.95,
        decay=0,
        epsilon=None
    ),
    loss='mse' # loss = "binary_crossentropy" or "mse"?
)

# train it
autoencoder.fit(
    x_train, x_train,
    epochs=10,
    batch_size=128,
    shuffle=True,
    validation_data=(x_test, x_test)
)

Epoch 1/10
43/43 [==============================] - 1s 12ms/step - loss: 0.0335 - val_loss: 0.0306
Epoch 2/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0292 - val_loss: 0.0289
Epoch 3/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0280 - val_loss: 0.0275
Epoch 4/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0269 - val_loss: 0.0266
Epoch 5/10
43/43 [==============================] - 0s 6ms/step - loss: 0.0260 - val_loss: 0.0260
Epoch 6/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0253 - val_loss: 0.0254
Epoch 7/10
43/43 [==============================] - 0s 6ms/step - loss: 0.0246 - val_loss: 0.0246
Epoch 8/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0240 - val_loss: 0.0241
Epoch 9/10
43/43 [==============================] - 0s 6ms/step - loss: 0.0234 - val_loss: 0.0234
Epoch 10/10
43/43 [==============================] - 0s 5ms/step - loss: 0.0230 - val_loss: 0.0227


time: 5 s


In [ ]:
# predict train and test datasets
pred_train = autoencoder.predict(x_train, verbose=1)
pred_test = autoencoder.predict(x_test, verbose=1)

43/43 [==============================] - 0s 2ms/step
time: 649 ms


In [ ]:
# display decoded image
if dataset == 1:
    x = pred_test[23].reshape(28,28)
    plt.imshow(x)

time: 1.41 ms


In [ ]:
def extract_flatten(model, data, layer):
    # extracts and flatten the output of the encoder
    encoder_out = model.layers[layer].output
    encoder_out_flat = Flatten()(encoder_out)

    # creates a model that will return these outputs given the model input
    model_feat = Model(inputs=model.input, outputs=encoder_out_flat)

    # predict the features
    features = model_feat.predict(data)

    print("features.shape = ", features.shape)
    
    return features

time: 4.28 ms


In [ ]:
# extract features
feat_train = extract_flatten(autoencoder, x_train, features_layer)
feat_test = extract_flatten(autoencoder, x_test, features_layer)

features.shape =  (5400, 1024)
features.shape =  (1350, 1024)
time: 451 ms


In [ ]:
# standardize/normalize features
feat_train, data_mean, data_std, data_max, data_min = norm_stand(feat_train, None, None, None, None)
feat_test, data_mean, data_std, data_max, data_min = norm_stand(feat_test, data_mean, data_std, data_max, data_min)

--- normalize and standardize data ---
data_max =  0.17580275237560272
data_min =  0.0
---
new_data_max =  1.0
new_data_min =  0.0
new_data_mean =  0.02806861212294918
new_data_std =  0.06514685215945959


--- normalize and standardize data ---
data_max =  0.17580275237560272
data_min =  0.0
---
new_data_max =  1.007479621624925
new_data_min =  0.0
new_data_mean =  0.02867590781169114
new_data_std =  0.06656897141028191


time: 126 ms


In [ ]:
# features dataset
x_tr = np.copy(feat_train[:train_data,:])
index_tr = np.copy(index_train[:train_data])
x_lb = np.copy(feat_train[:label_data,:])
index_lb = np.copy(index_train[:label_data])
x_ts = np.copy(feat_test[:test_data,:])
index_ts = np.copy(index_test[:test_data])

time: 14.9 ms


In [ ]:
# hyper-parameters
map_wth = 16
map_hgt = 16
#input_dim = 784
#class_nbr = 5
nbr_epochs = 10
eps_i_list = [1.0]
eps_f_list = [0.01]
eta_i_list = [10.0]
eta_f_list = [0.01]
sigma_kernel = 1.0

time: 2.66 ms


In [ ]:
# hyper-parameters grid search
hyper_param_list, accuracy_list = [], []
for eps_i in eps_i_list:
    for eps_f in eps_f_list:
        for eta_i in eta_i_list:
            for eta_f in eta_f_list:
                hyper_param_list.append([eps_i, eps_f, eta_i, eta_f])
                weights, accuracy = run_som(eps_i, eps_f, eta_i, eta_f, x_tr, x_lb, index_lb, x_ts, index_ts)
                accuracy_list.append(accuracy)


hyper-parameters:   # eps_i = 1.000000   # eps_f = 0.010000   # eta_i = 10.000000   # eta_f = 0.010000
---------- epoch 1 ----------
---------- epoch 2 ----------
---------- epoch 3 ----------
---------- epoch 4 ----------
---------- epoch 5 ----------
---------- epoch 6 ----------
---------- epoch 7 ----------
---------- epoch 8 ----------
---------- epoch 9 ----------
---------- epoch 10 ----------
mnist test accuracy = 44.67
time: 44.3 s


In [ ]:
# best hyper-parameters
best_accuracy = np.max(accuracy_list)
best_hyper_param = hyper_param_list[np.argmax(accuracy_list)]
print("best accuracy = ", best_accuracy)
print("best hyper-parameters:   # eps_i = %f   # eps_f = %f   # sig_i = %f   # sig_f = %f" % (best_hyper_param[0], best_hyper_param[1], best_hyper_param[2], best_hyper_param[3]))

best accuracy =  44.666666666666664
best hyper-parameters:   # eps_i = 1.000000   # eps_f = 0.010000   # sig_i = 10.000000   # sig_f = 0.010000
time: 4.01 ms


In [ ]:
# run labeling
label_data = 600
run_labeling(weights, class_nbr, label_data, x_tr, index_tr, x_ts, index_ts)

mnist test accuracy = 43.56
mnist test accuracy = 39.33
mnist test accuracy = 40.07
mnist test accuracy = 39.33
mnist test accuracy = 41.33
mnist test accuracy = 39.19
mnist test accuracy = 40.89
mnist test accuracy = 40.96
mnist test accuracy = 39.56
mnist test accuracy = 42.96
time: 20.4 s


In [ ]:
# GPU memory check
gpu_report()

"""
# display neurons weights as mnist digits
som_grid = plt.figure(figsize=(10, 10)) # width, height in inches
for n in range(map_wth*map_hgt):
    image = weights[n].reshape([80,80]) # x_train[num] is the 784 normalized pixel values
    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)
    sub.set_axis_off()
    clr = sub.imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")
    #plt.colorbar(clr)
"""

CPU RAM Free: 25.0 GB
GPU 0 ... Mem Free: 15463MB / 16280MB | Utilization   5%


'\n# display neurons weights as mnist digits\nsom_grid = plt.figure(figsize=(10, 10)) # width, height in inches\nfor n in range(map_wth*map_hgt):\n    image = weights[n].reshape([80,80]) # x_train[num] is the 784 normalized pixel values\n    sub = som_grid.add_subplot(map_wth, map_hgt, n + 1)\n    sub.set_axis_off()\n    clr = sub.imshow(image, cmap = plt.get_cmap("jet"), interpolation = "nearest")\n    #plt.colorbar(clr)\n'

time: 67.9 ms
